Эта тетрадка содержит примеры и упражнения по двум основным методам векторизации текста: Bag of Words (мешок слов) и TF-IDF (Term Frequency-Inverse Document Frequency).

In [1]:
import re
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

**Часть 1: Исходные данные**

In [ ]:
# Наш корпус документов
documents = [
    "Машинное обучение - это подраздел искусственного интеллекта.",
    "Нейронные сети широко используются в машинном обучении.",
    "Глубокое обучение основано на многослойных нейронных сетях.",
    "Искусственный интеллект имитирует когнитивные функции человека.",
    "Компьютерное зрение и обработка языка - примеры применения ИИ."
]

# Вывод документов с номерами
for i, doc in enumerate(documents, 1):
    print(f"Документ {i}: {doc}")

**Часть 2: Предобработка**

In [ ]:
for i in range(len(documents)):
    # Приведение к нижнему регистру
    documents[i] = documents[i].lower()
    # Удаление пунктуации
    documents[i] = re.sub(r'[^\w\s]', '', documents[i])

  # Вывод предобработанных документов
print("Предобработанные документы:")
for i, doc in enumerate(documents, 1):
    print(f"Документ {i}: {doc}")

**Часть 3: Модель Bag of Words (мешок слов)**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  # Для создания Bag of Words

3.1 Создание матрицы Bag of Words

In [ ]:
# Создание векторизатора
count_vectorizer = CountVectorizer()

# Обучение векторизатора и преобразование документов
bow_matrix = count_vectorizer.fit_transform(documents)

# Получение списка фичей (слов)
feature_names = count_vectorizer.get_feature_names_out()

# Преобразование разреженной матрицы в плотную для наглядности
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=feature_names,
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы Bag of Words
print("Матрица Bag of Words:")
print(bow_df)

Получение списка фичей (слов)

feature_names = count_vectorizer.get_feature_names_out()
Что происходит:
Этот шаг извлекает из обученного CountVectorizer все уникальные слова (или n-граммы), которые были использованы при создании матрицы Bag of Words.

Зачем это нужно:
* Интерпретация результатов: Без этого шага у нас была бы только числовая матрица, но мы не знали бы, какой столбец соответствует какому слову.
* Понимание словаря: Метод get_feature_names_out() возвращает список всех уникальных терминов, которые векторизатор извлек из документов и включил в свой словарь.
* Подготовка к визуализации: Имена признаков необходимы для создания понятной таблицы с метками строк и столбцов.

**Часть 4: Модель TF-IDF**

4.1 Создание матрицы TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Создание TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer()

# Обучение векторизатора и преобразование документов
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы TF-IDF
print("Матрица TF-IDF:")
print(tfidf_df)

**Часть 5. Сравнение результатов**

In [ ]:
import numpy as np

In [ ]:
# Суммируем значения по всем документам для BOW
# Для BOW просто суммируем частоты слов по всем документам
bow_sum = np.sum(bow_matrix.toarray(), axis=0)

# Создаем словарь слово -> суммарная частота
word_bow_dict = dict(zip(feature_names, bow_sum))

# Находим топ-3 слова по BOW (самые частотные в коллекции)
top_bow_words = sorted(word_bow_dict.items(), key=lambda x: x[1], reverse=True)[:3]

print("=== Топ-3 слова по всей коллекции ===\n")
print("Топ-3 слова по BOW (самые частые в коллекции):")
for word, count in top_bow_words:
    print(f"- {word}: {count} раз")

# Для TF-IDF нужно суммировать значения по всем документам
tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)

# Создаем словарь слово -> суммарный TF-IDF вес
word_tfidf_dict = dict(zip(feature_names, tfidf_sum))

# Находим топ-3 слова по TF-IDF (с наибольшим суммарным весом по всей коллекции)
top_tfidf_words = sorted(word_tfidf_dict.items(), key=lambda x: x[1], reverse=True)[:3]

print("\nТоп-3 слова по TF-IDF (с наибольшим весом по всей коллекции):")
for word, score in top_tfidf_words:
    print(f"- {word}: {score:.4f}")

# Показываем разницу между списками
print("\nСравнение топ-3 слов:")
bow_words = [word for word, _ in top_bow_words]
tfidf_words = [word for word, _ in top_tfidf_words]

common_words = set(bow_words) & set(tfidf_words)
bow_only = set(bow_words) - common_words
tfidf_only = set(tfidf_words) - common_words

if common_words:
    print(f"Общие слова в обоих топ-3: {', '.join(common_words)}")
if bow_only:
    print(f"Только в BOW топ-3: {', '.join(bow_only)}")
if tfidf_only:
    print(f"Только в TF-IDF топ-3: {', '.join(tfidf_only)}")

**Самостоятельная работа**

ЗАДАНИЕ:
1. Предобработайте эти документы (удалите стоп-слова, приведите к нижнему регистру)
2. Создайте матрицу Bag of Words
3. Создайте матрицу TF-IDF
4. Найдите и выведите топ-3 важных слова для каждого документа по Bag of Words
5. Найдите и выведите топ-3 важных слова для каждого документа по TF-IDF
6. Проанализируйте разницу между результатами и объясните её


In [ ]:
# Создаем коллекцию документов
exercise_documents = [
    "Япония поражает сочетанием древних традиций и футуристических технологий. Токио с его небоскребами и неоновыми огнями соседствует с тихими храмами и садами. Весной цветение сакуры превращает страну в розовое облако. Японская кухня, от суши до рамена, предлагает уникальные вкусовые ощущения. Синкансэны позволяют комфортно перемещаться между городами.",

    "Исландия — страна потрясающих природных контрастов. Ледники соседствуют с действующими вулканами, а горячие гейзеры бьют среди снежных равнин. Северное сияние зимой и незаходящее солнце летом создают ощущение другой планеты. Голубая лагуна с ее геотермальными водами — идеальное место для расслабления после долгих пеших походов по национальным паркам.",

    "Таиланд привлекает путешественников белоснежными пляжами и кристально чистой водой. Острова Пхукет и Самуи предлагают роскошные курорты и активный отдых. Бангкок поражает контрастами: золотые храмы соседствуют с оживленными рынками и современными торговыми центрами. Тайская кухня с ее острыми ароматами и экзотическими фруктами — отдельное гастрономическое путешествие.",

    "Италия — настоящий музей под открытым небом. Рим хранит наследие древней империи в Колизее и Форуме. Венеция очаровывает каналами и гондолами. Флоренция — сокровищница искусства эпохи Возрождения. Побережье Амальфи и озеро Комо предлагают живописные пейзажи. Итальянская кухня, от пасты до джелато, заслуженно считается одной из лучших в мире.",

    "Перу хранит тайны древних цивилизаций. Мачу-Пикчу, затерянный город инков, привлекает туристов со всего мира. Линии Наска, гигантские рисунки на плато, до сих пор остаются загадкой. Озеро Титикака поражает своими плавучими островами, на которых живут местные племена. Перуанская кухня, с ее свежими морепродуктами и разнообразием картофеля, переживает всемирное признание.",

    "Марокко — это калейдоскоп красок и ароматов. Медины Феса и Марракеша с их узкими улочками и шумными базарами погружают в атмосферу арабских сказок. Пустыня Сахара предлагает незабываемые ночи под звездами в берберских лагерях. Атласские горы привлекают любителей трекинга. Марокканская кухня славится тажинами и кус-кусом, приправленными экзотическими специями.",

    "Новая Зеландия — рай для любителей природы и активного отдыха. Фьорды Милфорд-Саунд, ледники Южных Альп и гейзеры Роторуа демонстрируют разнообразие ландшафтов. Хоббитон, декорации из фильмов «Властелин колец», привлекают поклонников Толкина. Маори, коренное население, сохраняет свою уникальную культуру. Адреналиновые развлечения, от банджи-джампинга до рафтинга, доступны по всей стране.",

    "Кения предлагает классическое африканское сафари. Масаи-Мара, Амбосели и Цаво — национальные парки, где можно увидеть «большую пятерку» африканских животных в их естественной среде. Ежегодная миграция антилоп гну — одно из самых впечатляющих природных зрелищ. Пляжи Момбасы с коралловыми рифами идеальны для дайвинга и снорклинга. Племена масаи и самбуру сохраняют традиционный образ жизни.",

    "Вьетнам сочетает богатую историю и динамичное настоящее. Бухта Халонг с ее карстовыми островами — природное чудо. Хойан очаровывает древними улочками и бумажными фонариками. Дельта Меконга предлагает возможность познакомиться с сельской жизнью. Вьетнамская кухня, от фо до свежих спринг-роллов, покоряет своей свежестью. Система туннелей Ку-Чи напоминает о недавней войне.",

    "Антарктида — последний неосвоенный континент, привлекающий самых отважных путешественников. Круизы из Ушуаи позволяют увидеть айсберги, пингвинов и китов. Пересечение пролива Дрейка — настоящее испытание для морских путешественников. Исследовательские станции разных стран ведут научную работу в суровых условиях. Полуночное солнце летом создает сюрреалистичные пейзажи ледяной пустыни."
]

In [7]:
# ваш код
!pip install pandas pd
!pip install pandas scikit-learn numpy nltk
import re
import pandas as pd
from nltk.corpus import stopwords
import nltk
import numpy as np

nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")

exercise_documents = [
    "Япония поражает сочетанием древних традиций и футуристических технологий. Токио с его небоскребами и неоновыми огнями соседствует с тихими храмами и садами. Весной цветение сакуры превращает страну в розовое облако. Японская кухня, от суши до рамена, предлагает уникальные вкусовые ощущения. Синкансэны позволяют комфортно перемещаться между городами.",

    "Исландия — страна потрясающих природных контрастов. Ледники соседствуют с действующими вулканами, а горячие гейзеры бьют среди снежных равнин. Северное сияние зимой и незаходящее солнце летом создают ощущение другой планеты. Голубая лагуна с ее геотермальными водами — идеальное место для расслабления после долгих пеших походов по национальным паркам.",

    "Таиланд привлекает путешественников белоснежными пляжами и кристально чистой водой. Острова Пхукет и Самуи предлагают роскошные курорты и активный отдых. Бангкок поражает контрастами: золотые храмы соседствуют с оживленными рынками и современными торговыми центрами. Тайская кухня с ее острыми ароматами и экзотическими фруктами — отдельное гастрономическое путешествие.",

    "Италия — настоящий музей под открытым небом. Рим хранит наследие древней империи в Колизее и Форуме. Венеция очаровывает каналами и гондолами. Флоренция — сокровищница искусства эпохи Возрождения. Побережье Амальфи и озеро Комо предлагают живописные пейзажи. Итальянская кухня, от пасты до джелато, заслуженно считается одной из лучших в мире.",

    "Перу хранит тайны древних цивилизаций. Мачу-Пикчу, затерянный город инков, привлекает туристов со всего мира. Линии Наска, гигантские рисунки на плато, до сих пор остаются загадкой. Озеро Титикака поражает своими плавучими островами, на которых живут местные племена. Перуанская кухня, с ее свежими морепродуктами и разнообразием картофеля, переживает всемирное признание.",

    "Марокко — это калейдоскоп красок и ароматов. Медины Феса и Марракеша с их узкими улочками и шумными базарами погружают в атмосферу арабских сказок. Пустыня Сахара предлагает незабываемые ночи под звездами в берберских лагерях. Атласские горы привлекают любителей трекинга. Марокканская кухня славится тажинами и кус-кусом, приправленными экзотическими специями.",

    "Новая Зеландия — рай для любителей природы и активного отдыха. Фьорды Милфорд-Саунд, ледники Южных Альп и гейзеры Роторуа демонстрируют разнообразие ландшафтов. Хоббитон, декорации из фильмов «Властелин колец», привлекают поклонников Толкина. Маори, коренное население, сохраняет свою уникальную культуру. Адреналиновые развлечения, от банджи-джампинга до рафтинга, доступны по всей стране.",

    "Кения предлагает классическое африканское сафари. Масаи-Мара, Амбосели и Цаво — национальные парки, где можно увидеть «большую пятерку» африканских животных в их естественной среде. Ежегодная миграция антилоп гну — одно из самых впечатляющих природных зрелищ. Пляжи Момбасы с коралловыми рифами идеальны для дайвинга и снорклинга. Племена масаи и самбуру сохраняют традиционный образ жизни.",

    "Вьетнам сочетает богатую историю и динамичное настоящее. Бухта Халонг с ее карстовыми островами — природное чудо. Хойан очаровывает древними улочками и бумажными фонариками. Дельта Меконга предлагает возможность познакомиться с сельской жизнью. Вьетнамская кухня, от фо до свежих спринг-роллов, покоряет своей свежестью. Система туннелей Ку-Чи напоминает о недавней войне.",

    "Антарктида — последний неосвоенный континент, привлекающий самых отважных путешественников. Круизы из Ушуаи позволяют увидеть айсберги, пингвинов и китов. Пересечение пролива Дрейка — настоящее испытание для морских путешественников. Исследовательские станции разных стран ведут научную работу в суровых условиях. Полуночное солнце летом создает сюрреалистичные пейзажи ледяной пустыни."
]

documents = []
for text in exercise_documents:
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = [w for w in text.split() if w not in russian_stopwords]
    documents.append(" ".join(words))

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
bow_matrix = count_vectorizer.fit_transform(documents)
feature_names = count_vectorizer.get_feature_names_out()

bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=feature_names,
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

print("Матрица Bag of Words:")
print(bow_df)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

print("Матрица TF-IDF:")
print(tfidf_df)

bow_sum = np.sum(bow_matrix.toarray(), axis=0)
word_bow_dict = dict(zip(feature_names, bow_sum))
top_bow_words = sorted(word_bow_dict.items(), key=lambda x: x[1], reverse=True)[:3]

print("Топ-3 слова по BOW:")
for word, count in top_bow_words:
    print(f"- {word}: {count} раз")

tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)
word_tfidf_dict = dict(zip(feature_names, tfidf_sum))
top_tfidf_words = sorted(word_tfidf_dict.items(), key=lambda x: x[1], reverse=True)[:3]

print("\nТоп-3 слова по TF-IDF:")
for word, score in top_tfidf_words:
    print(f"- {word}: {score:.4f}")

print("\nСравнение топ-3 слов:")
bow_words = [word for word, _ in top_bow_words]
tfidf_words = [word for word, _ in top_tfidf_words]

common_words = set(bow_words) & set(tfidf_words)
bow_only = set(bow_words) - common_words
tfidf_only = set(tfidf_words) - common_words

if common_words:
    print(f"Общие слова в обоих топ-3: {', '.join(common_words)}")
if bow_only:
    print(f"Только в BOW топ-3: {', '.join(bow_only)}")
if tfidf_only:
    print(f"Только в TF-IDF топ-3: {', '.join(tfidf_only)}")
     


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.17.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB 119.1 kB/s eta 0:01:08
   ---------------------------------------- 0.0/8.1 MB 119.1 kB/s eta 0:01:08
   ---------------------------------------- 0.0/8.1 MB 122.9 kB/s eta 0:01:06
   ---------------------------------------- 0.1/8.1 MB 172.4 kB/s eta 0:00:47
   ---------------------------------------- 0.1/8.1 MB 218.5 kB/s eta 0:00:37
   ---------------------------------------- 0.1/


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shvet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Матрица Bag of Words:
             адреналиновые  айсберги  активного  активный  альп  амальфи  \
Документ 1               0         0          0         0     0        0   
Документ 2               0         0          0         0     0        0   
Документ 3               0         0          0         1     0        0   
Документ 4               0         0          0         0     0        1   
Документ 5               0         0          0         0     0        0   
Документ 6               0         0          0         0     0        0   
Документ 7               1         0          1         0     1        0   
Документ 8               0         0          0         0     0        0   
Документ 9               0         0          0         0     0        0   
Документ 10              0         1          0         0     0        0   

             амбосели  антарктида  антилоп  арабских  ...  цивилизаций  \
Документ 1          0           0        0         0  ...          

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 163.8 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.0 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 216.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB 1.3 MB/s eta 0:00:07
   ---------------------------------------- 0.0/8.1 MB 960.0 kB/s eta 0:00:09
   ---------------------------------------- 0.0/8.1 MB 960.0 kB/s eta 0:00:09
   ---------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\shvet\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\scipy\\_cyutility.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
